In [1]:
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from src.models import EndiannessModel, LinearRegression
from src.dataset_loaders import (
    ISAdetectEndiannessCountsDataset,
    random_train_test_split,
)

num_epochs = 2
learning_rate = 0.0005
batch_size = 4
SEED = random.randint(0, 1000)

dataset = ISAdetectEndiannessCountsDataset(
    dataset_path="../../dataset/ISAdetect/ISAdetect_full_dataset",
)
train_set, test_set = random_train_test_split(dataset, test_split=0.2, seed=SEED)

# print(f"og {id(dataset)},train {id(train_set.dataset)}, test {id(test_set.dataset)}") # all the same

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=1)

### Define model and train


In [2]:
model = EndiannessModel(with_sigmoid=False)
model = LinearRegression(4, 1, with_sigmoid=False)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

endianness_map = {"little": 0, "big": 1}

print("Training model...")
# Training loop
for epoch in range(num_epochs):
    print(f"before enumerate {epoch}")
    for i, (batch_x, batch_y) in enumerate(train_loader):
        if i == 0:
            print(f"starting_epoch {epoch}")
        optimizer.zero_grad()
        output = model(batch_x)
        targets = torch.tensor(
            [endianness_map[e] for e in batch_y["endianness"]], dtype=torch.float32
        )
        loss = criterion(output, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

        # if epoch == 0 and i < 10:
        #     print(f"Initial loss: {loss.item()}")

        if (i + 1) % (len(train_loader) // 10) == 0:
            print(f"Step {i+1}, Loss: {loss.item()}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Training model...
before enumerate 0
starting_epoch 0
Step 1927, Loss: 2.275108814239502
Step 3854, Loss: 0.1582840085029602
Step 5781, Loss: 0.24069735407829285
Step 7708, Loss: 0.0014561244752258062
Step 9635, Loss: 0.20799168944358826
Step 11562, Loss: 0.2718775272369385
Step 13489, Loss: 0.3446406126022339
Step 15416, Loss: 0.24330419301986694
Step 17343, Loss: 0.28450343012809753
Step 19270, Loss: 0.07750675827264786
Epoch 1/2, Loss: 0.004390288610011339
before enumerate 1
starting_epoch 1
Step 1927, Loss: 0.28967106342315674
Step 3854, Loss: 0.007202850189059973
Step 5781, Loss: 0.0
Step 7708, Loss: 0.005396324675530195
Step 9635, Loss: 0.12786294519901276
Step 11562, Loss: 1.4603112958866404e-06
Step 13489, Loss: 0.3621600270271301
Step 15416, Loss: 0.06898842751979828
Step 17343, Loss: 0.3739725351333618
Step 19270, Loss: 0.25658074021339417
Epoch 2/2, Loss: 0.038231778889894485


### Test model on .code only


In [3]:
# Test model on code only
dataset.use_code_only = True
print(f"{dataset.use_code_only=}")
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    arch_stats = {}
    j = 0
    for batch_x, batch_y in test_loader:
        if j == 0:
            print(f"{batch_x[0]=}")
            j += 1
        output = model(batch_x)
        batch_y_endian = [endianness_map[e] for e in batch_y["endianness"]]
        batch_y_arch = batch_y["architecture"]
        for i in range(len(output)):
            if output[i] >= 0.5:
                pred = 1
            else:
                pred = 0

            current_arch = batch_y_arch[i]
            if current_arch not in arch_stats:
                arch_stats[current_arch] = {"correct": 0, "total": 0}

            if pred == batch_y_endian[i]:
                correct += 1
                arch_stats[current_arch]["correct"] += 1

            arch_stats[current_arch]["total"] += 1
            total += 1

    overall_accuracy = correct / total
    print(f"\nOverall Accuracy: {overall_accuracy:.4f} ({correct}/{total})")

    # Print per-architecture accuracies
    print("\nPer-Architecture Accuracies:")
    for arch in sorted(arch_stats.keys()):
        arch_correct = arch_stats[arch]["correct"]
        arch_total = arch_stats[arch]["total"]
        arch_accuracy = arch_correct / arch_total
        print(f"{arch:10s}: {arch_accuracy:.4f} ({arch_correct}/{arch_total})")

dataset.use_code_only=True
batch_x[0]=tensor([53.,  3.,  0.,  3.])

Overall Accuracy: 0.8925 (17207/19279)

Per-Architecture Accuracies:
alpha     : 1.0000 (806/806)
amd64     : 0.9989 (871/872)
arm64     : 1.0000 (701/701)
armel     : 1.0000 (798/798)
armhf     : 0.9474 (739/780)
hppa      : 0.0740 (75/1014)
i386      : 1.0000 (1018/1018)
ia64      : 0.9990 (1011/1012)
m68k      : 0.9840 (863/877)
mips      : 0.6717 (485/722)
mips64el  : 0.9932 (879/885)
mipsel    : 0.9948 (762/766)
powerpc   : 0.7264 (547/753)
powerpcspe: 0.8882 (707/796)
ppc64     : 0.9961 (512/514)
ppc64el   : 1.0000 (709/709)
riscv64   : 0.9704 (851/877)
s390      : 0.8931 (911/1020)
s390x     : 0.9887 (700/708)
sh4       : 0.9568 (1152/1204)
sparc     : 0.8097 (804/993)
sparc64   : 0.7673 (488/636)
x32       : 1.0000 (818/818)


In [4]:
# Test model on full program
dataset.use_code_only = False
print(f"{dataset.use_code_only=}")
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    arch_stats = {}
    j = 0
    for batch_x, batch_y in test_loader:
        if j == 0:
            print(f"{batch_x[0]=}")
            j += 1
        output = model(batch_x)
        batch_y_endian = [endianness_map[e] for e in batch_y["endianness"]]
        batch_y_arch = batch_y["architecture"]
        for i in range(len(output)):
            if output[i] >= 0.5:
                pred = 1
            else:
                pred = 0

            current_arch = batch_y_arch[i]
            if current_arch not in arch_stats:
                arch_stats[current_arch] = {"correct": 0, "total": 0}

            if pred == batch_y_endian[i]:
                correct += 1
                arch_stats[current_arch]["correct"] += 1

            arch_stats[current_arch]["total"] += 1
            total += 1

    overall_accuracy = correct / total
    print(f"\nOverall Accuracy: {overall_accuracy:.4f} ({correct}/{total})")

    # Print per-architecture accuracies
    print("\nPer-Architecture Accuracies:")
    for arch in sorted(arch_stats.keys()):
        arch_correct = arch_stats[arch]["correct"]
        arch_total = arch_stats[arch]["total"]
        arch_accuracy = arch_correct / arch_total
        print(f"{arch:10s}: {arch_accuracy:.4f} ({arch_correct}/{arch_total})")

dataset.use_code_only=False
batch_x[0]=tensor([158.,  51.,   0.,   6.])

Overall Accuracy: 0.9873 (19034/19279)

Per-Architecture Accuracies:
alpha     : 1.0000 (806/806)
amd64     : 0.9931 (866/872)
arm64     : 1.0000 (701/701)
armel     : 0.9987 (797/798)
armhf     : 0.9808 (765/780)
hppa      : 0.9931 (1007/1014)
i386      : 0.8517 (867/1018)
ia64      : 0.9980 (1010/1012)
m68k      : 1.0000 (877/877)
mips      : 0.9972 (720/722)
mips64el  : 0.9989 (884/885)
mipsel    : 0.9948 (762/766)
powerpc   : 0.9987 (752/753)
powerpcspe: 0.9962 (793/796)
ppc64     : 1.0000 (514/514)
ppc64el   : 1.0000 (709/709)
riscv64   : 0.9761 (856/877)
s390      : 1.0000 (1020/1020)
s390x     : 1.0000 (708/708)
sh4       : 0.9859 (1187/1204)
sparc     : 0.9930 (986/993)
sparc64   : 0.9890 (629/636)
x32       : 1.0000 (818/818)
